# MS-E2122 - Nonlinear Optimization
Prof. Fabricio Oliveira

## Project Assignment 2 - ADMM

In [1]:
using JuMP
using Ipopt
using Random
using Test
using LinearAlgebra

The code below generates the structure used to create and give instances as input to the routines we will implement later.

In [2]:
struct Instance
    I  # Supplier index set
    J  # Client index set
    S  # Scenario index set
    C  # Unit capacity costs of suppliers
    D  # Client demands in all scenarios
    Q  # Unit costs of unfulfilled demand
    P  # Scenario probabilities
    F  # Unit costs to fulfil demands
    Bs # Max supplier capacities
    Bc # Max budget (cost) for capacity acquisition
end

function generate_instance(nI, nJ, nS)
    Random.seed!(1)
    I = 1:nI                          # Supplier index set
    J = 1:nJ                          # Client index set
    S = 1:nS                          # Scenario index set
    C = rand(5:20, nI)                # Unit capacity costs of suppliers
    D = rand(nJ,nS).*rand(5:40, nJ)   # Client demands in all scenarios
    Q = rand(4000:90000, nJ)          # Unit costs of unfulfilled demand
    P = ones(nS).*1/nS                # Scenario probabilities
    F = rand(2:30, (nI,nJ))           # Unit costs to fulfil demands
    Bs = rand(10:80, nI)              # Max supplier capacities
    Bc = 1500                         # Max budget (cost) for capacity acquisition

    Instance(I, J, S, C, D, Q, P, F, Bs, Bc)
end

function unpack_instance(instance)
    return (instance.I, 
            instance.J, 
            instance.S, 
            instance.C, 
            instance.D, 
            instance.Q, 
            instance.P, 
            instance.F, 
            instance.Bs, 
            instance.Bc
            )
end

function print_solution_stats(x, C)
    # Print capacity cost.
    fval = dot(C, value.(x))   # Optimal cost of reserved capacities                        
    println("Optimal cost of reserved capacities: ", fval)

    # Print optimal solution 
    println("Optimal solution:")
    for i = 1:length(x)
        println("x[$i] = ", round(value(x[i]),digits=2))
    end
end

print_solution_stats (generic function with 1 method)

Here is the implementaiton of the stochastic capacity expansion problem as described. You can use this as reference for benchmarking the correctness and the performance of your model.

In [3]:
# Function to solve an instance of the full-scale model
function full_scale_model(instance; log=false)
    
    # Unpacking instance information
    (I, J, S, C, D, Q, P, F, Bs, Bc) = unpack_instance(instance)

    # We first solve the problem formulation directly without ADMM
    model = Model(Ipopt.Optimizer)                      # We use Ipopt solver to compare with ADMM                                                       

    # Variables
    @variable(model, x[i in I] >= 0)                    # Reserved capacity variables
    @variable(model, y[i in I, j in J, s in S] >= 0)    # Demand fulfilment variables
    @variable(model, u[j in J, s in S] >= 0)            # Unfulfilled demand variables

    # Objective: Minimize the total phase 1 + phase 2 costs over all scenarios
    @objective(model, Min,
          sum(C[i]*x[i] for i in I) +
          sum(P[s]*F[i,j]*y[i,j,s] for s in S, i in I, j in J) + 
          sum(P[s]*Q[j]*u[j,s] for s in S, j in J))

    # Constraints
    @constraint(model, [i in I], x[i] <= Bs[i])          # Max capacity constraint
    @constraint(model, sum(C[i]*x[i] for i in I) <= Bc)  # Max capacity budget (cost) constraint

    # Capacity reserve limit constraint for each supplier i in each scenario s
    @constraint(model, [i in I, s in S], sum(y[i,j,s] for j in J) <= x[i])

    # Demand balance constraint (u[j,s] is unfulfilled demand of client j in scenario s)
    @constraint(model, DemBal[j in J, s in S], sum(y[i,j,s] for i in I) + u[j,s] == D[j,s])
 
    if !log
        set_silent(model)    # Solve the problem
    end
    
    optimize!(model)         # Solve the problem
    
    print_solution_stats(x, C)
    return value.(x) 
    
end

full_scale_model (generic function with 1 method)

We can then generate one "test" instance that will serve as reference for us.

In [4]:
Random.seed!(1)
@time test_instance = generate_instance(15,20,10);

  0.000050 seconds (19 allocations: 23.461 KiB)


To time these functions, we will use the `@time` macro since the operation takes several seconds and doing multiple replications would be time consuming. Notice however that Julia uses JIT compilation, so the first time the function is run, it is also compiled. To discount this time from the timing, you cansimply remove the percentage associated with compilation time. 

Of course, once you ran the function on this test instance, it will have been compiled and you don't need then to worry about compilation times when considering the actual instances later.

In [5]:
@time x_full = full_scale_model(test_instance);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Optimal cost of reserved capacities: 1500.000016297421
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 12.0
x[11] = 0.0
x[12] = 0.0
x[13] = 29.2
x[14] = 0.0
x[15] = 39.0
 42.297379 seconds (94.09 M allocations: 5.462 GiB, 5.07% gc time, 38.80% compilation time)


## ADMM implementation

Next we will implement the function that employs ADMM to solve the stochastic capacity expansion problem. Notice you are required to implement:
1. The objective function of the augmented Lagrangian form
2. The combined primal and dual residual
3. The z-update
4. The dual update.

The input of the function is an `Instance` struct. The output is the value of the JuMP variable `x_s` and the number of iterations `k`.

In [6]:
function admm_model(instance; ρ=1.0, N=200, ϵ=1e-2, stat=true)   
    
    # Unpacking instance information
    (I, J, S, C, D, Q, P, F, Bs, Bc) = unpack_instance(instance)
    nI = length(I) 
    nJ = length(J)
    nS = length(S)

    # NOTE: ADMM approach starts from here. Complete the missing code parts that
    #       are requested. NOTE: Use Exercise 10.1 as a reference.

    # Problem parameters and memory allocation. NOTE: Compare with Exercise 10.1
    x_s  = zeros(nI, nS)  # Store each x vector in (x,y)-steps of each scenario
    v_s  = zeros(nI, nS)  # Store each v vector in v-steps of each scenario
    z    = zeros(nI)      # Store z vectors at each ADNM iteration
    residual = 0          # Store the value of residual
     
    # Main loop
    for k = 1:N
        # Loop for solving each (x,y) step separately for each scenario s in S.
        for s in S   

            # Model to solve (x,y)-step of the current scenario subproblem s
            scen_m = Model(optimizer_with_attributes(Ipopt.Optimizer)) 
            
            @variable(scen_m, x[i in I] >= 0)                    # Reserved capacity variables
            @variable(scen_m, y[i in I, j in J] >= 0)            # Demand fulfilment variables
            @variable(scen_m, u[j in J] >= 0)                    # Unfulfilled demand variables

            @constraint(scen_m, [i in I], x[i] <= Bs[i])          # Max capacity constraint
            @constraint(scen_m, sum(C[i]*x[i] for i in I) <= Bc)  # Max capacity budget (cost)
            
            # Capacity reserve limit constraint for each supplier i
            @constraint(scen_m, [i in I], sum(y[i,j] for j in J) <= x[i])
            
            # Demand balance constraint (u[j] is unfulfilled demand of client j)
            @constraint(scen_m, [j in J], sum(y[i,j] for i in I) == D[j,s] - u[j])

            # TODO: Complete this objective to compute (x,y)-step for the
            #       current scenario. 
            @objective(scen_m, Min, 
            P[s]*(
                sum((C[i] + v_s[i,s])*x[i] for i in I) + 
                sum(F[i,j]*y[i,j] for j in J, i in I) + 
                sum(Q[j]*u[j] for j in J) + 
                ρ/2 .* sum((x[i] - z[i])^2 for i in I)))             
            
            # Solve the (x,y) step for the current scenario
            set_silent(scen_m)
            optimize!(scen_m) 

            # Store the value of x for the current scenario
            x_s[:,s] = value.(x[:])
        end

        # Compute primal and dual residuals. We use array to exploit parallelism
        tol = zeros(nS)
        
        # TODO: Complete the computation of the residual for each s.
        for s in S
            tol[s] = P[s]*ρ*norm(x_s[:,s] - z)
        end
        
        # Total residual = sum of subproblem residuals
        tol = sum(tol[s] for s in S)
        residual = tol
        
        # Print current progress
        if stat
            println("iteration: $k | residual: $tol")
        end
        
        # Stopping condition: if primal + dual residual is small enough
        if tol < 1e-1
            if stat
                print_solution_stats(sum(x_s,dims=2)./nS, C)
            end
            
            finval = dot(C, value.(sum(x_s,dims=2)./nS))
            return (finval, value.(x_s), k, residual)
        end

        # TODO: Compute z-step for this iteration
        z = sum(P[s] * x_s[:,s] for s in S)

        # TODO: Update v-step separately for each scenario
        for s in S
            v_s[:,s] = v_s[:,s] + ρ*(x_s[:,s] - z)
        end
        
        # Check whether the iterations limit is exceeded while stopping condition isn't met
        if k == N 
            println("Algorithm terminated because of exceeding the limit of iterations($N))")
        end
    end
    
    if stat
        print_solution_stats(sum(x_s,dims=2)./nS, tend, C)
    end
    
    finval = dot(C, value.(sum(x_s,dims=2)./nS))
    
    return (finval, value.(x_s), k, residual) 
end

admm_model (generic function with 1 method)

As before, let us test our implementation on a test instance. For verifying our implementation, we will compare the solution of the full scale model and the ADMM model to verify correctness.

In [7]:
@time (value_test, x_s, k, residual_test) = admm_model(test_instance);

iteration: 1 | residual: 83.73554150422136
iteration: 2 | residual: 4.096208260493523
iteration: 3 | residual: 0.4919742516462082
iteration: 4 | residual: 0.1835077236123957
iteration: 5 | residual: 0.09740995330556428
Optimal cost of reserved capacities: 1499.9220248164238
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 11.96
x[11] = 0.0
x[12] = 0.0
x[13] = 29.23
x[14] = 0.0
x[15] = 39.0
  9.532685 seconds (5.29 M allocations: 328.240 MiB, 1.40% gc time, 40.00% compilation time)


In [8]:
#=The actual solution is the average of the scenario-dependent solutions (with 10 scenarios for the test instance). Notice that they should all match is the
primal residual is sufficiently small=#
x_admm = sum(x_s, dims=2)./10

println("residual value: $residual_test")
println("optimal value: $value_test")

# Test whether the solutions match, within a specific tolerance
@test norm(x_full - x_admm) ≤ 0.1
@test residual_test ≤ 0.1

residual value: 0.09740995330556428
optimal value: 1499.9220248164238


Test Passed

We are now ready to make comparisons considering different instances (with 50, 75 and 100 scenarios). Make sure you have executed your functions before in the test instance, so Julia JIT compiler has the function ready and you can measure only execution time. (Notice that the time macro will inform any compiling time otherwise).

Use instances below to perform your computational experiments. Use the small instances to try different values of $\rho \in [0.5 , 100]$. Some ideas for analyses include:
1. How does the value of $\rho$ influence the precision (which can be measured by `norm(x_full - x_admm)` of the final solution)?
2. What value of $\rho$ provides the fastest time (tip: you can use the macro `@elapsed` to save the time to a variable. E.g.)
```julia
t = @elapsed full_scale_model(small_instance)
print(t) # Prints out the time taken by the function, now saved on variable t.
```
3. Is ADMM faster? How does the difference between the solution times of the full scale and the ADMM models behaves as the number of scenarios increase?

In [9]:
small_instance = generate_instance(15,20,50)
medium_instance = generate_instance(15,20,75)
large_instance = generate_instance(15,20,100);

## Additional code for the Project assignment 2

Use the cells below or any additional cell you might need to write the code to generate the analytical results required.

In [11]:
# test results obtained by ADMM vs Ipopt on small instance
@time x_full_s = full_scale_model(small_instance);
@time (value_small, xas, ks, residual_small) = admm_model(small_instance);

println("residual value: $residual_small")
println("optimal value: $value_small")

x_admm_s = sum(xas, dims=2)./50
@test norm(x_full_s - x_admm_s) ≤ 0.1
@test residual_small ≤ 0.1

Optimal cost of reserved capacities: 1500.0000162994563
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 12.0
x[11] = 0.0
x[12] = 0.0
x[13] = 29.2
x[14] = 0.0
x[15] = 39.0
 28.277555 seconds (298.68 k allocations: 27.029 MiB)
iteration: 1 | residual: 85.37410570048952
iteration: 2 | residual: 1.6900427204808692
iteration: 3 | residual: 0.3470692311728298
iteration: 4 | residual: 0.14094416039284693
iteration: 5 | residual: 0.06523412263161721
Optimal cost of reserved capacities: 1499.9163503848993
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.01
x[10] = 11.98
x[11] = 0.0
x[12] = 0.0
x[13] = 29.19
x[14] = 0.0
x[15] = 38.99
 32.103224 seconds (2.42 M allocations: 205.902 MiB, 0.60% gc time)
residual value: 0.06523412263161721
optimal value: 1499.9163503848993


Test Passed

In [12]:
# test results obtained by ADMM vs Ipopt on medium instance
@time x_full_m = full_scale_model(medium_instance);
@time (value_medium, xam, km, residual_medium) = admm_model(medium_instance);

println("residual value: $residual_medium")
println("optimal value: $value_medium")

x_admm_m = sum(xam, dims=2)./75
@test norm(x_full_m - x_admm_m) ≤ 0.1
@test residual_medium ≤ 0.1

Optimal cost of reserved capacities: 1500.0000162996562
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 12.0
x[11] = 0.0
x[12] = 0.0
x[13] = 29.2
x[14] = 0.0
x[15] = 39.0
 56.497712 seconds (445.19 k allocations: 40.788 MiB, 0.07% gc time)
iteration: 1 | residual: 85.36213738480843
iteration: 2 | residual: 1.9327076451477392
iteration: 3 | residual: 0.3987162676929186
iteration: 4 | residual: 0.1479343984310342
iteration: 5 | residual: 0.05239057845652937
Optimal cost of reserved capacities: 1499.8997739751876
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.01
x[10] = 11.98
x[11] = 0.0
x[12] = 0.0
x[13] = 29.19
x[14] = 0.0
x[15] = 38.99
 51.226362 seconds (3.63 M allocations: 308.927 MiB, 0.71% gc time)
residual value: 0.05239057845652937
optimal value: 1499.8997739751876


Test Passed

In [13]:
# test results obtained by ADMM vs Ipopt on large instance
@time x_full_l = full_scale_model(large_instance);
@time (value_large, xal, kl, residual_large) = admm_model(large_instance, ρ=100.0);

println("residual value: $residual_large")
println("optimal value: $value_large")

x_admm_l = sum(xal, dims=2)./100
@test norm(x_full_l - x_admm_l) ≤ 0.1
@test residual_large ≤ 0.1

Optimal cost of reserved capacities: 1500.0000162997326
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 12.0
x[11] = 0.0
x[12] = 0.0
x[13] = 29.2
x[14] = 0.0
x[15] = 39.0
106.633092 seconds (591.42 k allocations: 50.308 MiB, 0.03% gc time)
iteration: 1 | residual: 8413.16239589931
iteration: 2 | residual: 589.2417393844521
iteration: 3 | residual: 29.52691754733241
iteration: 4 | residual: 21.64544548501699
iteration: 5 | residual: 3.190531420635208
iteration: 6 | residual: 4.019767734041083e-6
Optimal cost of reserved capacities: 1500.0000162168465
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 43.0
x[9] = 0.0
x[10] = 12.0
x[11] = 0.0
x[12] = 0.0
x[13] = 29.2
x[14] = 0.0
x[15] = 39.0
 71.410976 seconds (5.80 M allocations: 493.428 MiB, 0.28% gc time, 0.02% compilation time)
residual value: 4.019767734041083e-6
optimal value: 1500.0000162168465


Test Passed

In [20]:
ρ_vals = [0.5, 0.7, 0.9, 1, 5, 7, 10, 20, 50, 70, 100]
w = length(ρ_vals) + 1
h = 15

15

In [15]:
num_instance_small = 50

x_solutions_small = zeros(h, w)
iterations_small = zeros(w)
times_small = zeros(w)
f_solutions_small = zeros(w)
residuals_small = zeros(w)

for (index, ρs) in enumerate(ρ_vals)
    println("--- ρ: $ρs ---")
    (val_sol_s, x_sol_s, iter_s, r_s) = admm_model(small_instance, ρ=ρs)
    
    x_vec_s = sum(x_sol_s, dims=2)./num_instance_small
    
    x_solutions_small[:, index] = x_vec_s
    iterations_small[index] = iter_s
    f_solutions_small[index] = val_sol_s
    residuals_small[index] = r_s
    
    t_s = @elapsed admm_model(small_instance, stat=false)
    times_small[index] = t_s
end

--- ρ: 0.5 ---
iteration: 1 | residual: 42.70104572910347
iteration: 2 | residual: 0.9233204818371603
iteration: 3 | residual: 0.22245104440928734
iteration: 4 | residual: 0.16552371661608878
iteration: 5 | residual: 0.11155132797744645
iteration: 6 | residual: 0.08842718416009045
Optimal cost of reserved capacities: 1498.935502750618
Optimal solution:
x[1] = 0.01
x[2] = 0.0
x[3] = 37.98
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.97
x[9] = 0.01
x[10] = 11.95
x[11] = 0.0
x[12] = 0.0
x[13] = 29.16
x[14] = 0.01
x[15] = 38.96
--- ρ: 0.7 ---
iteration: 1 | residual: 59.77037657620024
iteration: 2 | residual: 1.2333310248162428
iteration: 3 | residual: 0.23946616603213366
iteration: 4 | residual: 0.15003911251989321
iteration: 5 | residual: 0.09620646404571526
Optimal cost of reserved capacities: 1499.6477693012894
Optimal solution:
x[1] = 0.01
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.99
x[9] = 0.01
x[10] = 11.96
x[11] = 0.0
x[12] = 0.0
x[1

In [16]:
@time _ = full_scale_model(small_instance, log=true);

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    16000
Number of nonzeros in inequality constraint Jacobian.:    15780
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    16015
                     variables with only lower bounds:    16015
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1000
Total number of inequality constraints...............:      766
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      766

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [19]:
#println(iterations_small')
println(times_small')
#println(f_solutions_small')
println(residuals_small')

[32.821099506 34.233458115 33.698850291 34.642663653 31.988260941 34.459498734 35.579368243 29.079779791 34.724603386 35.016591223 30.521137219 0.0]
[0.08842718416009045 0.09620646404571526 0.07081732381222341 0.06523412263161721 0.04830292758687171 0.04258116107659493 0.057400953172343984 4.054972137079768e-6 5.563162421367969e-6 4.537914160776523e-6 3.892273907836654e-6 0.0]


In [21]:
num_instance_medium = 75

x_solutions_medium = zeros(h, w)
iterations_medium = zeros(w)
times_medium = zeros(w)
f_solutions_medium = zeros(w)
residuals_medium = zeros(w)

for (index, ρs) in enumerate(ρ_vals)
    println("--- ρ: $ρs ---")
    (val_sol_m, x_sol_m, iter_m, r_m) = admm_model(medium_instance, ρ=ρs)
    
    x_vec_m = sum(x_sol_m, dims=2)./num_instance_medium
    
    x_solutions_medium[:, index] = x_vec_m
    iterations_medium[index] = iter_m
    f_solutions_medium[index] = val_sol_m
    residuals_medium[index] = r_m
    
    t_m = @elapsed admm_model(medium_instance, stat=false)
    times_medium[index] = t_m
end

--- ρ: 0.5 ---
iteration: 1 | residual: 42.69728730130734
iteration: 2 | residual: 1.0645042711747446
iteration: 3 | residual: 0.2614362912009892
iteration: 4 | residual: 0.19196608845910326
iteration: 5 | residual: 0.15216615179245976
iteration: 6 | residual: 0.13436775834217643
iteration: 7 | residual: 0.08374534438257059
Optimal cost of reserved capacities: 1499.5148271478627
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.99
x[9] = 0.01
x[10] = 11.97
x[11] = 0.0
x[12] = 0.0
x[13] = 29.18
x[14] = 0.0
x[15] = 38.98
--- ρ: 0.7 ---
iteration: 1 | residual: 59.76266705222745
iteration: 2 | residual: 1.4112920715373487
iteration: 3 | residual: 0.31439167418026015
iteration: 4 | residual: 0.20256955309587688
iteration: 5 | residual: 0.11574950642486591
iteration: 6 | residual: 0.04994807898138466
Optimal cost of reserved capacities: 1499.9161602329987
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 38.0
x[4] = 37.0
x[5] = 15.0
x[

In [22]:
@time _ = full_scale_model(medium_instance, log=true);

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    24000
Number of nonzeros in inequality constraint Jacobian.:    23655
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    24015
                     variables with only lower bounds:    24015
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1500
Total number of inequality constraints...............:     1141
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     1141

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [23]:
#println(iterations_small')
println(times_medium)
#println(f_solutions_small')
println(residuals_medium)

[50.195682825, 53.480413602, 51.757510149, 52.767006981, 49.673227601, 51.047302423, 52.862016946, 52.824268659, 55.926891951, 51.878579117, 52.111245769, 0.0]
[0.08374534438257059, 0.04994807898138466, 0.05777982610280212, 0.05239057845652937, 0.055558010822576505, 0.054687785096972676, 0.049308501517232844, 0.08816099389195692, 5.353396235078606e-6, 6.2467509303763754e-6, 4.077689421932943e-6, 0.0]


In [24]:
num_instance_large = 100

x_solutions_large = zeros(h, w)
iterations_large = zeros(w)
times_large = zeros(w)
f_solutions_large = zeros(w)
residuals_large = zeros(w)

for (index, ρs) in enumerate(ρ_vals)
    println("--- ρ: $ρs ---")
    (val_sol_l, x_sol_l, iter_l, r_l) = admm_model(large_instance, ρ=ρs)
    
    x_vec_l = sum(x_sol_l, dims=2)./num_instance_large
    
    x_solutions_large[:, index] = x_vec_l
    iterations_large[index] = iter_l
    f_solutions_large[index] = val_sol_l
    residuals_large[index] = r_l
    
    t_l = @elapsed admm_model(large_instance, stat=false)
    times_large[index] = t_l
end

--- ρ: 0.5 ---
iteration: 1 | residual: 42.58404429279076
iteration: 2 | residual: 1.3043399152986321
iteration: 3 | residual: 0.3163030601262684
iteration: 4 | residual: 0.21868703325473526
iteration: 5 | residual: 0.16514208874743858
iteration: 6 | residual: 0.12748726640327085
iteration: 7 | residual: 0.08199120353599502
Optimal cost of reserved capacities: 1499.5055873480308
Optimal solution:
x[1] = 0.01
x[2] = 0.0
x[3] = 37.98
x[4] = 36.99
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.99
x[9] = 0.01
x[10] = 11.96
x[11] = 0.0
x[12] = 0.0
x[13] = 29.18
x[14] = 0.01
x[15] = 38.98
--- ρ: 0.7 ---
iteration: 1 | residual: 59.60105268188259
iteration: 2 | residual: 1.7073649582115114
iteration: 3 | residual: 0.3605375161782442
iteration: 4 | residual: 0.21698270137786865
iteration: 5 | residual: 0.11518300864426048
iteration: 6 | residual: 0.057491823978344625
Optimal cost of reserved capacities: 1499.9334885286253
Optimal solution:
x[1] = 0.01
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15

In [25]:
@time _ = full_scale_model(large_instance, log=true);

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    32000
Number of nonzeros in inequality constraint Jacobian.:    31530
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    32015
                     variables with only lower bounds:    32015
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2000
Total number of inequality constraints...............:     1516
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     1516

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [29]:
#println(iterations_large)
println(times_large)
#println(f_solutions_large)
println(residuals_large)

[65.693756128, 67.564866409, 68.295243775, 66.729003189, 69.409419912, 68.065073634, 70.153517887, 69.364893204, 69.575695253, 67.347090547, 68.291820916, 0.0]
[0.08199120353599502, 0.057491823978344625, 0.06862682065952529, 0.06345505860745565, 0.04206116133905694, 0.04246058879028131, 0.04935454061713757, 0.09436180098682753, 3.6667637765534874e-6, 5.2037693572638615e-6, 4.019767734041083e-6, 0.0]


In [30]:
Xlarge_instance = generate_instance(15,20,130);

In [32]:
num_instance_Xlarge = 130

x_solutions_Xlarge = zeros(h, w)
iterations_Xlarge = zeros(w)
times_Xlarge = zeros(w)
f_solutions_Xlarge = zeros(w)
residuals_Xlarge = zeros(w)

for (index, ρs) in enumerate(ρ_vals)
    println("--- ρ: $ρs ---")
    (val_sol_xl, x_sol_xl, iter_xl, r_xl) = admm_model(Xlarge_instance, ρ=ρs)
    
    x_vec_xl = sum(x_sol_xl, dims=2)./num_instance_Xlarge
    
    x_solutions_Xlarge[:, index] = x_vec_xl
    iterations_Xlarge[index] = iter_xl
    f_solutions_Xlarge[index] = val_sol_xl
    residuals_Xlarge[index] = r_xl
    
    t_xl = @elapsed admm_model(Xlarge_instance, stat=false)
    times_Xlarge[index] = t_xl
end

--- ρ: 0.5 ---
iteration: 1 | residual: 42.469852148980095
iteration: 2 | residual: 1.4392819220104773
iteration: 3 | residual: 0.3587470278402048
iteration: 4 | residual: 0.21578996682569979
iteration: 5 | residual: 0.1585645998107321
iteration: 6 | residual: 0.11876073919858546
iteration: 7 | residual: 0.0797761714033194
Optimal cost of reserved capacities: 1499.4213280481072
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.98
x[4] = 36.99
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.99
x[9] = 0.01
x[10] = 11.97
x[11] = 0.01
x[12] = 0.0
x[13] = 29.18
x[14] = 0.0
x[15] = 38.97
--- ρ: 0.7 ---
iteration: 1 | residual: 59.44016029125299
iteration: 2 | residual: 1.8807459491616574
iteration: 3 | residual: 0.382588551722192
iteration: 4 | residual: 0.20425883507535572
iteration: 5 | residual: 0.11177243908307187
iteration: 6 | residual: 0.059893823400276165
Optimal cost of reserved capacities: 1499.9394140873515
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x

In [33]:
@time _ = full_scale_model(Xlarge_instance, log=true);

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    41600
Number of nonzeros in inequality constraint Jacobian.:    40980
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    41615
                     variables with only lower bounds:    41615
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2600
Total number of inequality constraints...............:     1966
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     1966

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [35]:
#println(iterations_large)
println(times_Xlarge)

[87.827787984, 77.005790226, 77.27183401, 84.809529353, 83.19836985, 72.665583884, 83.387999616, 78.393334852, 86.293252752, 86.414683022, 73.879518787, 0.0]


In [10]:
# Obrain convergence results when ρ varies from 0.5 to 100 with step size 5
ρ = vcat([0.5, 0.7, 0.9, 1], collect(range(5,100,step=5)))
time = zeros(length(ρ),3)
iter = zeros(length(ρ),3)

for (i,j) in enumerate(ρ)
    t1 = @elapsed x1 = admm_model(small_instance, ρ=j, stat=true);
    t2 = @elapsed x2 = admm_model(medium_instance, ρ=j, stat=true);
    t3 = @elapsed x3 = admm_model(large_instance, ρ=j, stat=true);
    time[i,1] = t1
    time[i,2] = t2
    time[i,3] = t3
    iter[i,1] = x1[3]
    iter[i,2] = x2[3]
    iter[i,3] = x3[3]
end

iteration: 1 | residual: 42.70104572910347
iteration: 2 | residual: 0.9233204818371603
iteration: 3 | residual: 0.22245104440928734
iteration: 4 | residual: 0.16552371661608878
iteration: 5 | residual: 0.11155132797744645
iteration: 6 | residual: 0.08842718416009045
Optimal cost of reserved capacities: 1498.935502750618
Optimal solution:
x[1] = 0.01
x[2] = 0.0
x[3] = 37.98
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 42.97
x[9] = 0.01
x[10] = 11.95
x[11] = 0.0
x[12] = 0.0
x[13] = 29.16
x[14] = 0.01
x[15] = 38.96
iteration: 1 | residual: 42.69728730130734
iteration: 2 | residual: 1.0645042711747446
iteration: 3 | residual: 0.2614362912009892
iteration: 4 | residual: 0.19196608845910326
iteration: 5 | residual: 0.15216615179245976
iteration: 6 | residual: 0.13436775834217643
iteration: 7 | residual: 0.08374534438257059
Optimal cost of reserved capacities: 1499.5148271478627
Optimal solution:
x[1] = 0.0
x[2] = 0.0
x[3] = 37.99
x[4] = 37.0
x[5] = 15.0
x[6] = 0.0
x[7] = 0.0
x[8] = 4

In [11]:
using Plots
plot(ρ, time[:,1], xaxis="ρ", yaxis="time", title="Small instance", label="ADMM")#, seriestype = :scatter)
savefig("si_t.pdf")
plot(ρ, iter[:,1], xaxis="ρ", yaxis="iterations", title="Small instance", label="ADMM")
savefig("si_i.pdf")

plot(ρ, time[:,2], xaxis="ρ", yaxis="time", title="Medium instance", label="ADMM")
savefig("mi_t.pdf")
plot(ρ, iter[:,2], xaxis="ρ", yaxis="iterations", title="Medium instance", label="ADMM")
savefig("mi_i.pdf")

plot(ρ, time[:,3], xaxis="ρ", yaxis="time", title="Large instance", label="ADMM")
savefig("li_t.pdf")
plot(ρ, iter[:,3], xaxis="ρ", yaxis="iterations", title="Large instance", label="ADMM")
savefig("li_i.pdf")